# Get data
```
mkdir -p /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
cd /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20201006-maclaren-fs-7.1-samseg-aseg-long/ .
```

## Split data into seperate subdirs
To faciliate processing
```
cd /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
mkdir cross
mkdir long
mkdir long-base
mv sub-??_run-?? ./cross/
mv sub-??_base ./long-base
mv sub* ./long/
```

and then rename the long dirs (gross)
```
cd ./long
for DIR_STEM in `ls -1|sed 's/\..*//'`; do  SUB_NUM=`echo $DIR_STEM|sed 's/sub-//'|sed 's/_run.*//'`; mv ${DIR_STEM}.long.sub-${SUB_NUM}_base ${DIR_STEM}; done
```

In [9]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# imports find_json_files(); load_json_file(); load_dataset(); load_fs_dataset()
from cmeds import *
# imports calc_cvs(); session_permute(); monte_carlo_perm_test
from test_retest import *

In [10]:
structs_of_interest = [
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Right-Lateral-Ventricle',
    'Right-Hippocampus',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Putamen'
]
# Add some lateral regions together so we can directly compare to table 1 in MacLaren et al.
regions = [             
            [ ['Left-Hippocampus', 'Right-Hippocampus'],'Hippocampus' ],
            [ ['Left-Lateral-Ventricle', 'Right-Lateral-Ventricle'],'Lateral-Ventricles' ],
            [ ['Left-Amygdala', 'Right-Amygdala',],'Amygdala' ],
            [ ['Left-Putamen', 'Right-Putamen'],'Putamen' ],
            [ ['Left-Caudate', 'Right-Caudate'],'Caudate' ],
          ]
drop_subjects = []
demofile = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/demographics.tsv'
datadir = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/long'
maclaren_fs_vol_aseg_df = load_fs_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])
maclaren_fs_vol_aseg_df = add_regions(maclaren_fs_vol_aseg_df,regions)

#maclaren_fs_vol_samseg_df = load_fssamseg_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])
#maclaren_fs_vol_samseg_df = add_regions(maclaren_fs_vol_samseg_df,regions)

Dropping the following subjects []


In [14]:
# The column name that holds session info in the demographics.tsv
session_col='session'
# The column name that holds subject info in the demographics.tsv
subject_col='subject_num'
structs_of_interest = ['Hippocampus', 'Lateral-Ventricles', 'Amygdala', 'Putamen', 'Caudate']


session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [1,2,3]

n = 1000
perm_test_reults_aseg = monte_carlo_perm_test(maclaren_fs_vol_aseg_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')
#perm_test_reults_samseg = monte_carlo_perm_test(maclaren_fs_vol_samseg_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')


Comparable to [Table 1 in MacLaren et al](https://www.nature.com/articles/sdata201437/tables/2)

In [15]:
perm_test_reults_aseg

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8699.919167,11752.483333,3700.818333,10110.419167,7624.127500
total-cov,1.120952,2.707812,1.355289,1.211011,0.846865
session-cov,1.152905,1.756832,1.444742,1.102368,0.854518
abs-diff-cov,0.031953,0.950980,0.089453,0.108644,0.007652
p-vals,0.677000,0.000000,0.342000,0.150000,0.896000
